<a href="https://colab.research.google.com/github/thamirests/oilDerivatives/blob/main/fuelSales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
# Importando a biblioteca os
import os
# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
!pip install -q findspark
import findspark
findspark.init('spark-3.1.2-bin-hadoop2.7')

In [ ]:
#instalando bibliotecas
!pip install pandas==1.2.0
!pip install --upgrade xlrd
!pip install pyspark
!pip install xls2xlsx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 7.1 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 3.7 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 24.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=d32417386cb7527fc2caa09d410212a9ca126023ea49d13274013bbf0b728695
  Stored in direct

In [ ]:
from tables import table
import pandas as pd
import numpy as np
import xlrd
import pyspark as ps
from openpyxl import load_workbook
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

In [ ]:
#transformando xls em xlsx
p.save_book_as(file_name='/content/drive/MyDrive/Temp/Challenge_raizen/vendas-combustiveis-m3.xls',
               dest_file_name='/content/drive/MyDrive/Temp/Challenge_raizen/vendas-combustiveis-m3_1.xlsx')

In [ ]:
#extraindo e mudando formato
xls = "/content/drive/MyDrive/Temp/Challenge_raizen/vendas-combustiveis-m3.xlsx"
oilDerivatives = pd.read_excel(xls, sheet_name="DPCache_m3")
productUf=pd.melt(oilDerivatives, id_vars =['ANO','COMBUSTÍVEL', 'ESTADO'], value_vars =['Jan', 'Fev', 'Mar','Abr', 'Mai', 'Jun', 'Ago', 'Set', 'Out', 'Nov', 'Dez'],var_name ='Mes', value_name ='Volume')

In [ ]:
#diesel
diesel=oilDerivatives[oilDerivatives['COMBUSTÍVEL'].str.contains("DIESEL")]
diesel=diesel.pivot_table(values = 'TOTAL', index = 'COMBUSTÍVEL', columns = 'ESTADO', aggfunc='sum')

In [ ]:
#formatando dataframe
productUf['Unit'] = productUf['COMBUSTÍVEL'].apply(lambda st: st[st.find("(")+1:st.find(")")])
productUf['year_month'] = productUf['ANO'].astype(str) +'/'+ productUf['Mes']
productUf.drop(['ANO','Mes'], axis=1, inplace=True)
productUf[['year_month','ESTADO','COMBUSTÍVEL', 'Unit', 'Volume']]

,year_month,ESTADO,COMBUSTÍVEL,Unit,Volume
0,2000/Jan,RONDÔNIA,GASOLINA C (m3),m3,136073.253000
1,2000/Jan,ACRE,GASOLINA C (m3),m3,3358.346000
2,2000/Jan,AMAZONAS,GASOLINA C (m3),m3,20766.918000
3,2000/Jan,RORAIMA,GASOLINA C (m3),m3,3716.032000
4,2000/Jan,PARÁ,GASOLINA C (m3),m3,29755.907000
...,...,...,...,...,...
49891,2020/Dez,RIO GRANDE DO SUL,GLP (m3),m3,64045.161232
49892,2020/Dez,MATO GROSSO DO SUL,GLP (m3),m3,16281.139493
49893,2020/Dez,MATO GROSSO,GLP (m3),m3,18321.987319
49894,2020/Dez,GOIÁS,GLP (m3),m3,46850.585145


In [ ]:
#salvando no bucket

sdf = session.write.parquet(
    "wasbs://<container-name>@<storage-account-name>.blob.core.windows.net/<prefix>"
)